# Hello Radiomics example: using the feature extractor to calculate features

This example shows how to use the radiomics package and the feature extractor.
The feature extractor handles preprocessing, and then calls the needed featureclasses to calculate the features.
It is also possible to directly instantiate the feature classes. However, this is not recommended for use outside debugging or development. For more information, see `helloFeatureClass`.

In [1]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor
import radiomics

## Setting up logging

Regulate verbosity of PyRadiomics (outputs to stderr)

In [2]:
# Regulate verbosity with radiomics.setVerbosity
# radiomics.setVerbosity(logging.INFO)  # Use logging.DEBUG for maximum output, default verbosity level = WARNING

Set up logging to a log file

In [3]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='testLog.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

## Getting the testcase

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [4]:
imageName, maskName = radiomics.getTestCase('brain1')

if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

## Initializing the feature extractor

#### Extraction Settings

In [5]:
# Use a parameter file, this customizes the extraction settings and also specifies the input image types to use and which features should be extracted
params = os.path.join('..', 'examples', 'exampleSettings', 'Params.yaml')

extractor = featureextractor.RadiomicsFeaturesExtractor(params)

In [6]:
# Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
# settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True

extractor = featureextractor.RadiomicsFeaturesExtractor(**settings)

#### Input images: applying filters

In [7]:
# By default, only 'Original' (no filter applied) is enabled. Optionally enable some image types:

# extractor.enableImageTypeByName('Wavelet')
# extractor.enableImageTypeByName('LoG', customArgs={'sigma':[3.0]})
# extractor.enableImageTypeByName('Square')
# extractor.enableImageTypeByName('SquareRoot')
# extractor.enableImageTypeByName('Exponential')
# extractor.enableImageTypeByName('Logarithm')

# Alternative; set filters in one operation 
# This updates current enabled image types, i.e. overwrites custom settings specified per filter. 
# However, image types already enabled, but not passed in this call, are not disabled or altered.

# extractor.enableImageTypes(Wavelet={}, LoG={'sigma':[3.0]})

print('Enabled input images:')
for imageType in extractor._enabledImagetypes.keys():
    print('\t' + imageType)

Enabled input images:
	Original


#### Feature classes: setting which feature(classes) need to be calculated

In [8]:
# Disable all classes
extractor.disableAllFeatures()

# Enable all features in firstorder
extractor.enableFeatureClassByName('firstorder')

# Alternative; only enable 'Mean' and 'Skewness' features in firstorder
# extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])

## Getting the docstrings of the active features

In [10]:
print('Active features:')
for cls, features in six.iteritems(extractor._enabledFeatures):
    if len(features) == 0:
        features = [f for f, deprecated in six.iteritems(extractor.getFeatureNames(cls)) if not deprecated]
    for f in features:
        print(f)
        print(getattr(extractor.featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

Active features:
10Percentile

    **5. 10th percentile**

    The 10\ :sup:`th` percentile of :math:`\textbf{X}`
    
90Percentile

    **6. 90th percentile**

    The 90\ :sup:`th` percentile of :math:`\textbf{X}`
    
Energy

    **1. Energy**

    .. math::
      \textit{energy} = \displaystyle\sum^{N}_{i=1}{(\textbf{X}(i) + c)^2}

    Here, :math:`c` is optional value, defined by ``voxelArrayShift``, which shifts the intensities to prevent negative
    values in :math:`\textbf{X}`. This ensures that voxels with the lowest gray values contribute the least to Energy,
    instead of voxels with gray level intensity closest to 0.

    Energy is a measure of the magnitude of voxel values in an image. A larger values implies a greater sum of the
    squares of these values.

    .. note::
      This feature is volume-confounded, a larger value of :math:`c` increases the effect of volume-confounding.
    
Entropy

    **3. Entropy**

    .. math::
      \textit{entropy} = -\displaystyle\

## Calculating the values of the active features

In [11]:
print('Calculating features')
featureVector = extractor.execute(imageName, maskName)

Calculating features


Disabling C extensions
Calculating features with label: 1
Loading image and mask
Adding additional extraction information
Adding image type "Original" with settings: {'distances': [1], 'verbose': True, 'force2Ddimension': 0, 'enableCExtensions': False, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'normalizeScale': 1, 'normalize': False, 'additionalInfo': True, 'padDistance': 5, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'label': 1, 'resegmentRange': None, 'minimumROIDimensions': 1}
Calculating features for original image
Computing firstorder
Computing glcm
calculate GLCM: 100%|██████████████████████████| 33/33 [00:00<00:00, 49.77it/s]


In [12]:
# Show output
for featureName in featureVector.keys():
    print('Computed %s: %s' % (featureName, featureVector[featureName]))

Computed general_info_BoundingBox: (162, 84, 11, 47, 70, 7)
Computed general_info_EnabledImageTypes: {'Original': {}}
Computed general_info_GeneralSettings: {'distances': [1], 'verbose': True, 'additionalInfo': True, 'enableCExtensions': False, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'minimumROIDimensions': 1, 'resegmentRange': None, 'padDistance': 5}
Computed general_info_ImageHash: 5c9ce3ca174f0f8324aa4d277e0fef82dc5ac566
Computed general_info_ImageSpacing: (0.7812499999999999, 0.7812499999999999, 6.499999999999998)
Computed general_info_MaskHash: 9dc2c3137b31fd872997d92c9a92d5178126d9d3
Computed general_info_Version: 1.3.0.post14+g4425f4b
Computed general_info_VolumeNum: 2
Computed general_info_VoxelNum: 4137
Computed original_firstorder_InterquartileRange: 253.0
Computed original_firstorder_Skewness: 0.27